In [9]:
from github import Github
import pickle
import pandas as pd
import pprint
pd.set_option('display.max_rows', 150)
pp = pprint.PrettyPrinter(depth=6)

In [4]:
#using an access token
g = Github("3eccda0bf7e78b97af1eb5b82ede22021331293c")
repo = g.get_repo("SonarSource/sonarqube")

In [5]:
#returned commits in a paginated list
commits = repo.get_commits()

In [26]:
commits_list = []
parents = []
i=0
for commit in commits:
    i+=1
    commits_list.append((commit.sha, commit.commit.message, commit.commit.author.date))
    for parent in commit.commit.parents:
        parents.append((commit.sha, parent.sha))

RateLimitExceededException: 403 {"message": "API rate limit exceeded for user ID 16037020.", "documentation_url": "https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting"}

In [16]:
len(commits_list)

501

In [18]:
# Create pandas df
parentsDf = pd.DataFrame(parents, columns=["commit_sha", "parent_sha"])
parentsDf

,commit_sha,parent_sha
0,6dc2cc9ff6340e6588b7608ed90354aa6767fcdf,0ecd8610552ccd59049308c1be46b03470c48538
1,0ecd8610552ccd59049308c1be46b03470c48538,bc413ee3db73258a22059816bd4766bcea66a9ca
2,bc413ee3db73258a22059816bd4766bcea66a9ca,27282ea04940d30eb33c20a02989a79d88dec4a0
3,27282ea04940d30eb33c20a02989a79d88dec4a0,626469caaa8b060f79f101cffff214d23c7e611b
4,626469caaa8b060f79f101cffff214d23c7e611b,dae0f392c72132be88af4a73d98875fdf477de50
...,...,...
496,88597f3717a873db1373e4d4b2bcca7f5fd74425,9ea4d1c0a65d82acd4c4d3628a82b6a673fcb0c9
497,9ea4d1c0a65d82acd4c4d3628a82b6a673fcb0c9,74384e67058fcfea16c80839740d28d9083b944a
498,74384e67058fcfea16c80839740d28d9083b944a,7294268682b6d1add416c7de5d41422367fec324
499,7294268682b6d1add416c7de5d41422367fec324,4f5231d18118c6643c19f1d1259f1165b2764774


In [24]:
(parentsDf.groupby("commit_sha").count().parent_sha == 1).sum()

501

In [80]:
# Create pandas df
commitsDf = pd.DataFrame(commits_list, columns=["Sha", "Message", "Date"])
commitsDf.Date = pd.to_datetime(commitsDf.Date)

In [81]:
commitsDf

,Sha,Message,Date
0,4b3d914bf578d1eb88fc9ca79efef00843511865,Upgrade Gradle plugins,2020-09-24 18:16:15
1,ad7d964270d6a6c3da52bf54931ce9fbf50c3076,SONAR-13905 upgrade Hazelcast from 3.12.7 to 3...,2020-09-24 08:38:51
2,552f5bbe1d4eca4783a4da1325b16b0cd4d7783f,SONAR-13905 upgrade Tomcat from 8.5.56 zo 8.5....,2020-09-24 08:13:42
3,9a9609134d2c5136baed9436f9f54eb20821a25e,SONAR-10558 Make it explicit that the project'...,2020-09-28 09:05:10
4,c87841b757f4a849ddae0195d3f0bb98e2d5223f,Fix issue with BBS repos fetch,2020-09-25 12:04:29
...,...,...,...
26995,3f099f35475a66e24290c29004cdc6acef49800d,Improve message when database must be upgraded,2012-03-26 08:57:03
26996,2796325b4b87e9b273f7b4fe0649ae3f6297a51b,SONAR-3354 Display empty table instead of mess...,2012-03-27 08:34:42
26997,c3d0fe433d773d015f2fb99521a82b8ab29c0da6,Fix violations,2012-03-27 07:37:20
26998,2d4f0cd2bb0b768f401ca776eb256d7f31957976,Improve DB readme file.,2012-03-27 07:36:25


In [14]:
pp.pprint(commits[0].commit.parents[0].sha)

'0ecd8610552ccd59049308c1be46b03470c48538'


In [18]:
pp.pprint(commits_list[0].__dict__)

{'_CompletableGithubObject__completed': False,
 '_author': <github.GithubObject._ValuedAttribute object at 0x111c77460>,
 '_comments_url': <github.GithubObject._ValuedAttribute object at 0x111c5b100>,
 '_commit': <github.GithubObject._ValuedAttribute object at 0x111c55580>,
 '_committer': <github.GithubObject._ValuedAttribute object at 0x111c58a90>,
 '_files': NotSet,
 '_headers': {'access-control-allow-origin': '*',
              'access-control-expose-headers': 'ETag, Link, Location, '
                                               'Retry-After, X-GitHub-OTP, '
                                               'X-RateLimit-Limit, '
                                               'X-RateLimit-Remaining, '
                                               'X-RateLimit-Used, '
                                               'X-RateLimit-Reset, '
                                               'X-OAuth-Scopes, '
                                               'X-Accepted-OAuth-Scopes, '
          

In [30]:
# Dump tags objects retrieved from Github APIs

tags = repo.get_tags()
f = open('pkl/tags.pkl', 'wb')
for tag in tags:
    pickle.dump(tag, f)

In [31]:
tmp = []
for tag in tags:
    tmp.append((tag.name, tag.commit.sha, repo.get_commit(sha=tag.commit.sha).commit.author.date))

In [27]:
# Create a file with tags pandas df
tagsDf = pd.DataFrame(tmp, columns=["Tag", "Sha", "Date"])
#tagsDf.to_csv("csv/tags.csv")

NameError: name 'tmp' is not defined

In [7]:
tagsDf = pd.read_csv("csv/tags.csv")
tagsDf.Date = pd.to_datetime(tagsDf.Date)

In [10]:
import datetime
tagsDf[tagsDf.Date > pd.to_datetime(datetime.date(2015,3,10))]

,Unnamed: 0,Tag,Sha,Date
1,1,8.4.2.36762,2de1f343e3dbd9e88007b42e475512677c04eb8a,2020-08-27 14:11:31
2,2,8.4.1.35646,a0d372f704998d38856c26e28662fd7de062f873,2020-07-14 08:13:18
3,3,8.4.0.35506,74b4533439f380827d68e8c01667eeaecad89fc7,2020-07-03 07:09:27
4,4,8.3.1.34397,b77ce511a2039330aea8832ad12e86a8cfa9ff7d,2020-05-07 12:09:22
5,5,8.3.0.34182,9c58e36328ae79ad3c36c13aa253879351ce3613,2020-04-28 13:54:13
6,6,8.2.0.32929,749367809478e5b7f60c6ef952436db9789b5882,2020-02-25 14:32:24
7,7,8.1.0.31237,65a7a7c34de5bff9aed7c3e58eb6dcc87ae34b57,2019-12-17 12:29:34
8,8,8.0,87ca68d63f4afd37d74b2f454430dfde9e862c6a,2019-10-15 08:15:12
9,9,7.9.4,2f8d99a55899105cbf3e3170194c614c2a6ea4d7,2020-07-23 14:40:10
10,10,7.9.3,898a79cc7df01a9603c17f92b93e46ec62bc77d2,2020-03-24 07:49:27


Get commits differences between two tags
https://stackoverflow.com/questions/40293833/how-to-get-all-commits-in-a-git-tag-through-github-api